# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [31]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

In [32]:
# Constants

MODEL = "llama3.2"

In [22]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [5]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [6]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI (Artificial Intelligence) has numerous business applications across various industries. Here are some examples:

1. **Content Generation**: Generative AI can create high-quality content such as articles, social media posts, product descriptions, and even entire books. This can help businesses save time and resources while maintaining consistency in their content.
2. **Image and Video Creation**: Generative AI can generate realistic images and videos that can be used for advertising, marketing, and entertainment purposes. For instance, AI-powered video creation tools can produce professional-quality explainer videos, product demos, or even social media clips.
3. **Chatbots and Virtual Assistants**: Generative AI can power chatbots and virtual assistants that provide customer support, answer frequently asked questions, and offer personalized recommendations. This can help businesses improve their customer experience and reduce the workload of human customer support agents.


# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [8]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Creation**: Generative AI can generate high-quality content, such as articles, social media posts, and product descriptions, at scale and speed. This can help businesses save time and resources on content creation.
2. **Image and Video Generation**: Generative AI can create realistic images and videos for use in advertising, marketing, and entertainment. For example, a company can generate 3D models of products for virtual try-on or create realistic product images for e-commerce websites.
3. **Chatbots and Virtual Assistants**: Generative AI can be used to develop more sophisticated chatbots and virtual assistants that can understand natural language and respond in a human-like manner.
4. **Predictive Analytics**: Generative AI can analyze large datasets and generate predictions about future trends and behaviors, helping businesses make data-driven decisions.
5. **Product D

## Alternative approach - using OpenAI python library to connect to Ollama

In [ ]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

## Are you confused about why that works?

It seems strange, right? We just used OpenAI code to call Ollama?? What's going on?!

Here's the scoop:

The python class `OpenAI` is simply code written by OpenAI engineers that makes calls over the internet to an endpoint.  

When you call `openai.chat.completions.create()`, this python code just makes a web request to the following url: "https://api.openai.com/v1/chat/completions"

Code like this is known as a "client library" - it's just wrapper code that runs on your machine to make web requests. The actual power of GPT is running on OpenAI's cloud behind this API, not on your computer!

OpenAI was so popular, that lots of other AI providers provided identical web endpoints, so you could use the same approach.

So Ollama has an endpoint running on your local box at http://localhost:11434/v1/chat/completions  
And in week 2 we'll discover that lots of other providers do this too, including Gemini and DeepSeek.

And then the team at OpenAI had a great idea: they can extend their client library so you can specify a different 'base url', and use their library to call any compatible API.

That's it!

So when you say: `ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')`  
Then this will make the same endpoint calls, but to Ollama instead of OpenAI.

## Also trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [ ]:
!ollama pull deepseek-r1:1.5b

In [ ]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [34]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

In [35]:
# Constants

MODEL = "llama3.2"

In [36]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped
    """
    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [37]:
# Let's try one out

ed = Website("https://campeones.com.ar/#/")
print(ed.title)
print(ed.text)

Portada | Campeones
Ir al contenido
Últimas Noticias
Nacionales
Internacionales
CALENDARIOS
CAMPEONATOS
Últimas Noticias
Nacionales
Internacionales
CALENDARIOS
CAMPEONATOS
Buscar
Lo más leído:
TC
Fórmula 1
TN
Procar
MotoGP
Top Race
TC
Fórmula 1
TN
Procar
MotoGP
Top Race
ACTIVIDAD DE FIN DE SEMANA
TN en San Juan: Lugón festejó su 1ª victoria en la Clase 3
27 abril, 2025
El cordobés se adjudicó la 4ª fecha de la división mayor del Turismo Nacional, disputada en El Villicum, tras liderar de principio a fin con el VW Virtus del equipo de Javier Merlo. Lo escoltaron el Corolla de Julián Santero, con 45 kilos de lastre, y el Cruze de Jorge Barrio, con 39 kilos extra.
TN C2: volvió Posco y volvió Ford a la victoria
TRV6: Rotundo triunfo de Felippo en San Nicolás
Llaver logró su primer triunfo en Turismo Carretera 2000
WRC: Rovanpera enciende el Mundial con una “masterclass” en Canarias
Campeones distinguido por la Legislatura de la Provincia de Buenos Aires
28 abril, 2025
NASCAR en Talladega:

In [39]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [40]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [41]:
# And now: call the Ollama function instead of OpenAI

def summarize(url):
    website = Website(url)
    messages = messages_for(website)
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']

In [42]:
summarize("https://campeones.com.ar/#/")

'¡Hola! Parece que estás buscando información sobre el mundo del automoción, específicamente sobre el Campeonato de Campeones. Sin embargo, la página web que has compartido parece ser una revista o un sitio web de noticias relacionadas con el deporte y la cultura en general, en lugar de un sitio web dedicado exclusivamente al Campeonato de Campeones.\n\nSi estás interesado en obtener información sobre un evento específico del Campeonato de Campeones, como resultados, clasificaciones o noticias, te recomendaría buscar en línea utilizando palabras clave relacionadas con el evento que deseas conocer. También podrías intentar visitar el sitio web oficial del Campeonato de Campeones, si existe.\n\nSin embargo, si estás interesado en obtener información sobre la revista Campeones, que parece ser un medio de comunicación que cubre temas como el automoción, el deporte y la cultura, puedo ofrecerte algunas sugerencias:\n\n* Puedes visitar el sitio web oficial de la revista Campeones para obtene

In [43]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [45]:
display_summary("https://campeones.com.ar/#/")

El piloto argentino, Mateos del Prete, logró un gran resultado en el Dakar 2025, llegando al final de la carrera con una destacada posición. Sin embargo, su historia se vio marcada por la adversidad y la lucha constante. Del Prete enfrentó múltiples desafíos durante su carrera, desde lesiones hasta problemas técnicos. A pesar de todo, su determinación y habilidad para adaptarse le permitieron mantenerse en el juego.

Del Prete comenzó con ambiciones altas y se dispuso a enfrentar el desafío más grande del año: la Carrera que Crea Leyendas. Con una gran cantidad de experiencia y preparación, se lanzó al rally con confianza y determinación. Sin embargo, su aventura en el Dakar 2025 fue interrumpida por la lesión que sufrió durante el event.

A pesar de esta adversidad, Mateos del Prete continuó adelante, demostrando una gran fortaleza mental y física. A lo largo del camino, también enfrentó desafíos técnicos y logísticos, pero nunca se rindió. Su determinación le permitió mantenerse en el juego, aunque su participación fue interrumpida por la lesión.

En su viaje hacia la victoria, Mateos del Prete también demostró una gran habilidad para adaptarse a las condiciones cambiantes y a los desafíos tecnológicos. Su experiencia y conocimiento de la vehículo le permitieron mantenerse enfocado en el objetivo principal: llegar al final de la carrera.

En resumen, Mateos del Prete tuvo un camino desafiante hacia la victoria en el Dakar 2025. A pesar de su lesión, demostró una gran determinación y adaptabilidad, manteniéndose enfocado en su objetivo principal: llegar al final de la carrera. Su historia es un ejemplo de la importancia de la perseverancia y la habilidad para superar obstáculos.